In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy.signal import butter
from scipy.signal import filtfilt
import re

def find_sync_file(mouse_file, animal_dat_path):
    ## find sync file: 
    for item in os.listdir(animal_dat_path):
        if mouse_file.split('_')[0] in item:
            if mouse_file.split('_')[1] == item[-1]:
                for folder in os.listdir(animal_dat_path + item):
                    if folder.split('_')[0].split('g')[-1] == mouse_file.split('_')[-1]:
                        current_animals_video_path = os.path.join(animal_dat_path + item,folder)+ r"\\video\tracking\\"
                        current_animals_behav_sync_path = os.path.join(animal_dat_path + item,folder)+ r"\\behav_sync\\"
                        for stage in os.listdir(current_animals_behav_sync_path):
                            if 'postsleep' in stage or 'post_sleep' in stage:
                                sync_path = current_animals_behav_sync_path + stage +r"\\"

    if mouse_file == '148_2_2':
        sync_df = pd.read_csv(sync_path + r'Postsleep_Ephys_Camera_sync.csv', encoding='cp1252')
    else:
        sync_df = pd.read_csv(sync_path + r'Postsleep_Ephys_Camera_sync.csv')
        
    return sync_df 

# new way based on Kime t al Ganguly
def butter_bandpass_filter(data, lowcut, highcut, fs, order_hp, order_lp):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist

    # High-pass filtering
    b_hp, a_hp = butter(order_hp, low, btype='high', analog=False)
    filtered_hp = filtfilt(b_hp, a_hp, data)

    # Low-pass filtering
    b_lp, a_lp = butter(order_lp, high, btype='low', analog=False)
    filtered_band = filtfilt(b_lp, a_lp, filtered_hp)

    return filtered_band

def extract_number(filename):
    """Extracts the numerical part from the filename."""
    match = re.search(r'channel-(\d+)', filename)
    if match:
        return int(match.group(1))
    return float('inf')  # Return infinity if no number is found



In [ ]:
### secondary processing of the LFP, to create actual LFP averages. 
experiment_type = "expert"
LFP_dat_path = r"Z:\projects\sequence_squad\revision_data\emmett_revisions\oscillations\striatum_lfp\\" + experiment_type + r'\\'

mouse_file = 'seq007_1_4'

# animal_dat_path = r'Z:\projects\sequence_squad\organised_data\animals\\'
animal_dat_path = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"

mouse_path = os.path.join(LFP_dat_path, mouse_file)
data_channels = []
channel_names = []
for channels in os.listdir(mouse_path):
    if 'striatum' in channels:
        print(channels)
        channel_path = os.path.join(mouse_path, channels)
        try:
            data_channels += [np.load(channel_path)]
        except:
            data_channels += [np.load(channel_path, allow_pickle=True)]
        channel_names +=[channels]
    elif 'timestamps' in channels:
        timestamps_df = pd.read_csv(os.path.join(mouse_path, channels))
   
# reorder them so that they are in channel order
sorted_inds = np.argsort([extract_number(item) for item in channel_names])
channel_names =  np.array(channel_names)[sorted_inds]
data_channels = np.array(data_channels)[sorted_inds]    


# hack to make sure that timestamps are correct length
if not len(data_channels[0]) == len(timestamps_df):
    timestamps_df = timestamps_df[0:len(data_channels[0])]
    print('timestamps and data channels are not the same length, truncating timestamps to match data channels')
            
### chop the data down to just the post sleep 

# load in the sync df to get the start time of the sleep period
sync_df = find_sync_file(mouse_file, animal_dat_path)
sleep_period_ephys_start_time = sync_df.Camera_time_Ephys_Aligned[0]
# make a mask for the sleep period
chopping_mask = timestamps_df.ephys_timestamp > sleep_period_ephys_start_time
# apply that mask to each channel and the timestamps
sleep_period_timestamps_df = timestamps_df[chopping_mask]
sleep_period_data_channels = []
for i,dat in enumerate(data_channels):
    sleep_period_data_channels += [dat[chopping_mask]]
